In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score

# Load the data
data = pd.read_csv('FL_eTime.csv')

# Prepare the data
y = data['Med_mac'].values
X = data.drop(['eTime_macos', 'Filename', 'Med_mac', 'Ave_mac', 'Std_mac'], axis=1)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the model
def create_model():
    model = Sequential([
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics = ['r2_score'])
    return model

# Train the model
model = create_model()
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
mse, r2_score = model.evaluate(X_test_scaled, y_test)
print(f"Model MSE: {mse}, R2:{r2_score}")

# Make predictions
predictions = model.predict(X_test_scaled)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0847 - r2_score: -10148717.0000 - val_loss: 0.0438 - val_r2_score: -6939397.0000
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0367 - r2_score: -4624951.0000 - val_loss: 0.0272 - val_r2_score: -4303560.0000
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0147 - r2_score: -1763668.1250 - val_loss: 0.0163 - val_r2_score: -2575472.5000
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0118 - r2_score: -1557710.8750 - val_loss: 0.0107 - val_r2_score: -1691033.8750
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0127 - r2_score: -1441926.0000 - val_loss: 0.0078 - val_r2_score: -1239532.1250
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0096 - r2_score: -1118365.7500 - val_loss: 0.0070 - val_r2_score: -1115562.6250
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0067 - r2_score: -894383.3750 - val_loss: 0.0062 - val_r2_score: -983602.8125
Epoch 8/100
4/4 ━━━━━━━━━━━

In [4]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

# 定义基学习器
base_learners = [
    ('xgb', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('svr', SVR(kernel='rbf', C=1, gamma=0.1))
]

# 定义堆叠回归模型
stacking_model = StackingRegressor(estimators=base_learners, final_estimator=RidgeCV())

# 训练模型
stacking_model.fit(X_train_scaled, y_train)

# 预测和评估模型
y_pred = stacking_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 9.32109127510933e-09
R-squared: -0.014779160386662715


LR

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 加载 CSV 文件
df = pd.read_csv('merged_results.csv')

# 特征选择
features = ['Ave_mac', 'eTime_macos', 'eTime_windows10', 'ReturnStmt', 'FieldDecl',
       'Class Defs', 'StructDecl', 'Static_Cast<>', 'FloatDecl',
       'MemberRefExpr', 'Max Depth', 'Ave_win', 'Med_win', 'BoolLiteral',
       'letdata']
X = df[features]
y = df['Med_mac']

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 模型训练
model = LinearRegression()
model.fit(X_train, y_train)

# 模型预测
y_pred = model.predict(X_test)

# 模型评估
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.4f}')
print(f'R-squared: {r2:.4f}')

RMSE: 0.0001
R-squared: 0.3083


/Users/vissan/anaconda3/envs/Cost/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RF

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 加载 CSV 文件
df = pd.read_csv('merged_results.csv')

# 选择 top_features
features = ['Ave_mac', 'eTime_macos', 'eTime_windows10', 'ReturnStmt', 'FieldDecl',
       'Class Defs', 'StructDecl', 'Static_Cast<>', 'FloatDecl',
       'MemberRefExpr', 'Max Depth', 'Ave_win', 'Med_win', 'BoolLiteral',
       'letdata']
X = df[features]
y = df['Med_mac']

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 模型训练
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 模型预测
y_pred = model.predict(X_test)

# 模型评估
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.4f}')
print(f'R-squared: {r2:.4f}')

RMSE: 0.0001
R-squared: 0.4255


/Users/vissan/anaconda3/envs/Cost/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


GBR

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 加载 CSV 文件
df = pd.read_csv('merged_results.csv')

# 选择 top_features
features = ['Ave_mac', 'eTime_macos', 'eTime_windows10', 'ReturnStmt', 'FieldDecl',
       'Class Defs', 'StructDecl', 'Static_Cast<>', 'FloatDecl',
       'MemberRefExpr', 'Max Depth', 'Ave_win', 'Med_win', 'BoolLiteral',
       'letdata']
X = df[features]
y = df['Med_mac']

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 模型训练
model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

# 模型预测
y_pred = model.predict(X_test)

# 模型评估
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.4f}')
print(f'R-squared: {r2:.4f}')

RMSE: 0.0001
R-squared: 0.3503


/Users/vissan/anaconda3/envs/Cost/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


NN

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, r2_score

# 加载 CSV 文件
df = pd.read_csv('merged_results.csv')

# 选择 top_features
features = ['Ave_mac', 'eTime_macos', 'eTime_windows10', 'ReturnStmt', 'FieldDecl',
       'Class Defs', 'StructDecl', 'Static_Cast<>', 'FloatDecl',
       'MemberRefExpr', 'Max Depth', 'Ave_win', 'Med_win', 'BoolLiteral',
       'letdata']
X = df[features]
y = df['Med_mac']

# 数据标准化
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 模型构建
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# 模型编译
model.compile(optimizer='adam', loss='mse')

# 模型训练
model.fit(X_train, y_train, epochs=100, verbose=0)

# 模型预测
y_pred = model.predict(X_test)

# 模型评估
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.4f}')
print(f'R-squared: {r2:.4f}')

/Users/vissan/anaconda3/envs/Cost/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
RMSE: 0.0695
R-squared: -526149.1716


/Users/vissan/anaconda3/envs/Cost/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
